In [1]:
%pip install pillow

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 2.8 MB/s eta 0:00:01
   ------------------------------- -------- 2.1/2.7 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pycuda

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy


  Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.5-cp312-cp312-win_amd64.whl (12.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pycuda.driver as cuda
import pycuda.autoinit

# Obtén el número de GPUs disponibles
device_count = cuda.Device.count()
print(f"Dispositivos CUDA disponibles: {device_count}")

# Imprime detalles de cada dispositivo CUDA
for i in range(device_count):
    device = cuda.Device(i)
    print(f"\nInformación de la tarjeta gráfica {i}:")
    print(f"  Nombre: {device.name()}")
    print(f"  Memoria total (GB): {device.total_memory() / (1024 ** 3):.2f} GB")
    print(f"  Arch: {device.compute_capability()}")
    print(f"  Multihilo: {device.get_attribute(cuda.device_attribute.MULTIPROCESSOR_COUNT)}")
    print(f"  Núcleos por multiprocesador: {device.get_attribute(cuda.device_attribute.MAX_THREADS_PER_MULTIPROCESSOR)}")
    print(f"  Frecuencia máxima (MHz): {device.get_attribute(cuda.device_attribute.CLOCK_RATE) / 1000}")
    print(f"  Unidad de procesamiento: {device.get_attribute(cuda.device_attribute.PCI_BUS_ID)}")


Dispositivos CUDA disponibles: 1

Información de la tarjeta gráfica 0:
  Nombre: NVIDIA GeForce RTX 4050 Laptop GPU
  Memoria total (GB): 6.00 GB
  Arch: (8, 9)
  Multihilo: 20
  Núcleos por multiprocesador: 1536
  Frecuencia máxima (MHz): 2355.0
  Unidad de procesamiento: 1


In [3]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy

a = numpy.random.randn(5, 5)
a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
__global__ void doubles_matrix(float *a) {
    int idx = threadIdx.x + blockIdx.y*5;
    a[idx] *= 2;
}
""")

func = mod.get_function("doubles_matrix")
func(a_gpu, block=(5, 5, 1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print("Matriz original:")
print(a)
print("DOUBLE MATRIX AFTER PyCUDA EXECUTION:")
print(a_doubled)

CompileError: nvcc compilation of C:\Users\Anthony\AppData\Local\Temp\tmpyvefs01h\kernel.cu failed
[command: nvcc --cubin -arch sm_89 -m64 -Ic:\Users\Anthony\Desktop\Entornos Virtuales Python\env-Pycuda\Lib\site-packages\pycuda\cuda kernel.cu]
[stdout:
nvcc fatal   : Cannot find compiler 'cl.exe' in PATH
]